# 定量評価

スペクトラルクラスタリングの結果を定量評価する．評価方法は以下．

- 花弁の数の比較
- IoU計算（除去した分岐点を除く）

#### データセットの格納場所
`adjacency_matrix/dataset`

### ライブラリのインポート

In [1]:
import cv2
import numpy as np
import sys
import random
from tqdm import tqdm

### データセットの画像番号指定
現在，11枚の画像を作成

In [ ]:
img_list = [600,613,624,658,665,673,694,700,710,720,730]

### 花弁の数を比較

In [ ]:
for img_num in tqdm(img_list):
    color_code_GT = np.zeros((200,3))
    color_code    = np.zeros((200,3))
    
    input_img_file = "Output/3D_inte/3D-inte_graph_Trans_reverse/ORA-{0:03d}_3D-inte_graph_Trans_reverse.png".format(img_num)
    input_dataset  = "dataset/annotation_NLMD/ORA-{0:03d}_mask_NLMD.png".format(img_num)

    img_my = cv2.imread(input_img_file)
    img_GT = cv2.imread(input_dataset)
    
    height,width,c = img_my.shape
    
    i = 1
    for y in range(height):
        for x in range(width):
            if((img_my[y,x,0]== img_my[y,x,1]) and (img_my[y,x,1] == img_my[y,x,2])):
                img_my[y,x] = (0,0,0)
            else:
                if(i==1):
                    color_code[i] = img_my[y,x]
                    i+=1
                    
                else:
                    flag = 0
                    i_cnt = 1

                    while(flag != 1 and i_cnt <= i):
                        if((color_code[i_cnt]==img_my[y,x]).all()):
                            flag = 1
                        else:
                            i_cnt += 1
                            
                    if(flag==0):
                        color_code[i] = img_my[y,x]
                        i+=1
                    
    j = 1
    for y in range(height):
        for x in range(width):
            if((img_GT[y,x,0]== img_GT[y,x,1]) and (img_GT[y,x,1] == img_GT[y,x,2])):
                img_GT[y,x] = (0,0,0)
            else:
                if(j==1):
                    color_code_GT[j] = img_GT[y,x]
                    j+=1
                    
                else:
                    flag = 0
                    j_cnt = 1

                    while(flag != 1 and j_cnt <= j):
                        if((color_code_GT[j_cnt]==img_GT[y,x]).all()):
                            flag = 1
                        else:
                            j_cnt += 1
                            
                    if(flag==0):
                        color_code_GT[j] = img_GT[y,x]
                        j+=1
                        
    print(img_num)
    print('My：{0:02d}, GT：{1:02d}'.format(i-1,j-1))

### IoU計算

In [ ]:
#for img_num in tqdm(img_list):
#input_img_file = "Output/3D_inte/3D-inte_graph_Trans_reverse/ORA-{0:03d}_3D-inte_graph_Trans_reverse.png".format(img_num)

img_num = 600

input_dataset  = "dataset/annotation_part/ORA-{0:03d}_mask_part.png".format(img_num)
input_result   = "dataset/test/ORA-{0:03d}_test.png".format(img_num)

input_original = "../../flower_CT_photo/ORA/[vg-data] ORA/volume_1/ORA-{0:03d}.tif".format(img_num)

img    = cv2.imread(input_original,0)
img_re = cv2.imread(input_result)
img_GT = cv2.imread(input_dataset)

img_NLMD = cv2.fastNlMeansDenoising(img,h=6)
th_img = np.where(img_NLMD>50 ,255,0)

height,width = img.shape

for y in range(height):
    for x in range(width):
        if(th_img[y,x]>0):
            if(img_re[y,x,0]==img_re[y,x,1] and img_re[y,x,2]==img_re[y,x,1]):
                img_re[y,x] = (0,0,255)
            elif((img_re[y,x]==img_GT[y,x]).all()):
                img_re[y,x] = (0,255,0)
            elif((img_re[y,x]!=img_GT[y,x]).all()):
                img_re[y,x] = (0,165,255)
        else:
            img_re[y,x] = (0,0,0)

cv2.imwrite('dataset/IoU/ORA-{0:03d}_IoU.png'.format(img_num),img_re)

In [ ]:
TP = 0
FP = 0
FN = 0

for y in range(height):
    for x in range(width):
        if((img_re[y,x]==(0,255,0)).all()):
            TP += 1
        elif((img_re[y,x]==(0,0,255)).all()):
            FN += 1
        elif((img_re[y,x]==(0,165,255)).all()):
            FP += 1
            
Precision = TP/(TP+FP)
Recall    = TP/(TP+FN)
Dice      = TP/(TP+((FP+FN)/2))
IoU       = TP/(TP+FP+FN)

print('適合率：{0:.3f}, 再現率：{1:.3f}, Dice係数：{2:.3f}, IoU：{3:.3f}'.format(Precision,Recall,Dice,IoU))

In [ ]:
print('TP：{0:03d}, FN：{1:03d}, FP：{2:03d}'.format(TP,FN,FP))

### 花弁ごとの検出率

In [ ]:
img_num = 700

input_dataset  = "dataset/annotation_part/ORA-{0:03d}_mask_part.png".format(img_num)
#input_dataset  = "dataset/annotation_NLMD/ORA-{0:03d}_mask_NLMD.png".format(img_num)
input_result   = "dataset/test/ORA-{0:03d}_test.png".format(img_num)

input_original = "../../flower_CT_photo/ORA/[vg-data] ORA/volume_1/ORA-{0:03d}.tif".format(img_num)

img    = cv2.imread(input_original,0)
img_re = cv2.imread(input_result)
img_GT = cv2.imread(input_dataset)

img_NLMD = cv2.fastNlMeansDenoising(img,h=6)
th_img = np.where(img_NLMD>50 ,255,0)

height,width = img.shape

#花弁の番号の割り当て
color_code_GT = np.zeros((200,3))

i = 1
for y in tqdm(range(height)):
    for x in range(width):
        if((img_GT[y,x,0]== img_GT[y,x,1]) and (img_GT[y,x,1] == img_GT[y,x,2])):
            img_GT[y,x] = (0,0,0)
        else:
            if(i==1):
                color_code_GT[i] = img_GT[y,x]
                i+=1

            else:
                flag = 0
                i_cnt = 1

                while(flag != 1 and i_cnt <= i):
                    if((color_code_GT[i_cnt]==img_GT[y,x]).all()):
                        flag = 1
                    else:
                        i_cnt += 1

                if(flag==0):
                    color_code_GT[i] = img_GT[y,x]
                    i+=1

#TP,FP,FNの割り当て
for y in tqdm(range(height)):
    for x in range(width):
        if(th_img[y,x]>0):
            if(img_re[y,x,0]==img_re[y,x,1] and img_re[y,x,2]==img_re[y,x,1]):
                #img_re[y,x] = (0,0,255)
                img_re[y,x] = img[y,x]
            elif((img_re[y,x]==img_GT[y,x]).all()):
                img_re[y,x] = (0,255,0)
            elif((img_re[y,x]!=img_GT[y,x]).all()):
                img_re[y,x] = (0,165,255)
        else:
            img_re[y,x] = (0,0,0)
            
mean_D_rate = 0

for petal_num in range(1,i):
    
    TP = 0
    FP = 0
    FN = 0
    
    y_s = 1000
    x_s = 1000
    y_e = 0
    x_e = 0
    
    for y in range(height):
        for x in range(width):
            if((img_GT[y,x] == color_code_GT[petal_num]).all()):
                if((img_re[y,x] == (0,255,0)).all()):
                    TP += 1
                elif((img_re[y,x] == (0,165,255)).all()):
                    FP += 1
                #elif((img_re[y,x] == (0,0,255)).all()):
                    #FN += 1
                    
                if(x_s > x):
                    x_s = x
                if(y_s > y):
                    y_s = y
                if(x_e < x):
                    x_e = x
                if(y_e < y):
                    y_e = y
    if(TP==0):
        D_rate = 0
    elif(TP>0):
        #D_rate = (TP/(TP+FP+FN))*100
        #mean_D_rate += (TP/(TP+FP+FN))
        D_rate = (TP/(TP+FP))*100
        mean_D_rate += (TP/(TP+FP))

    print('------------------------')
    print('petal_number:{0:d}'.format(petal_num))
    print('coordinates :({0:d},{1:d})-({2:d},{3:d})'.format(x_s,y_s,x_e,y_e))
    print('color code  :({0:d},{1:d},{2:d})'.format(int(color_code_GT[petal_num,2]),int(color_code_GT[petal_num,1]),int(color_code_GT[petal_num,0])))
    #print('TP:{0:d}, FP:{1:d}, FN:{2:d}, FP+FN:{3:d}, 検出率(FP+FN):{4:.1f}%'.format(TP,FP,FN,FP+FN,D_rate))
    print('TP:{0:d}, FP:{1:d}, 正解率:{2:.1f}%'.format(TP,FP,D_rate))
    
cv2.imwrite('dataset/IoU/ORA-{0:03d}_IoU_check.png'.format(img_num),img_re)
mean_D_rate = (mean_D_rate/(i-1))*100
print('------------------------')
print('平均正解率：{0:.1f}%'.format(mean_D_rate))

### 正解画像と花弁の対応付け

In [5]:
img_num = 658

#input_dataset  = "dataset/annotation_part/ORA-{0:03d}_mask_part.png".format(img_num)
input_dataset  = "dataset/annotation_NLMD/ORA-{0:03d}_mask_NLMD.png".format(img_num)
input_result   = "Output/3D_inte/3D-inte_graph_Trans_reverse/ORA-{0:03d}_3D-inte_graph_Trans_reverse.png".format(img_num)

input_original = "../../flower_CT_photo/ORA/[vg-data] ORA/volume_1/ORA-{0:03d}.tif".format(img_num)

img    = cv2.imread(input_original,0)
img_re = cv2.imread(input_result)
img_GT = cv2.imread(input_dataset)

img_NLMD = cv2.fastNlMeansDenoising(img,h=6)
th_img = np.where(img_NLMD>50 ,255,0)

height,width = img.shape

#花弁の番号の割り当て
color_code_GT = np.zeros((200,3))
color_code    = np.zeros((200,3))

code_temp_GT = np.zeros((height,width))
code_temp_re = np.zeros((height,width))

i = 1
for y in range(height):
    for x in range(width):
        if((img_re[y,x,0]== img_re[y,x,1]) and (img_re[y,x,1] == img_re[y,x,2])):
            img_re[y,x] = (0,0,0)
        else:
            if(i==1):
                color_code[i] = img_re[y,x]
                code_temp_re[y,x] = i
                i+=1

            else:
                flag = 0
                i_cnt = 1

                while(flag != 1 and i_cnt <= i):
                    if((color_code[i_cnt]==img_re[y,x]).all()):
                        flag = 1
                        code_temp_re[y,x] = i_cnt
                    else:
                        i_cnt += 1

                if(flag==0):
                    color_code[i] = img_re[y,x]
                    code_temp_re[y,x] = i
                    i+=1
                    
j = 1
for y in range(height):
    for x in range(width):
        if((img_GT[y,x,0]== img_GT[y,x,1]) and (img_GT[y,x,1] == img_GT[y,x,2])):
            img_GT[y,x] = (0,0,0)
        else:
            if(j==1):
                color_code_GT[j] = img_GT[y,x]
                code_temp_GT[y,x] = j
                j+=1

            else:
                flag = 0
                j_cnt = 1

                while(flag != 1 and j_cnt <= j):
                    if((color_code_GT[j_cnt]==img_GT[y,x]).all()):
                        flag = 1
                        code_temp_GT[y,x] = j_cnt
                    else:
                        j_cnt += 1

                if(flag==0):
                    color_code_GT[j] = img_GT[y,x]
                    code_temp_GT[y,x] = j
                    j+=1

#TP,FP,FNの割り当て
for result_num in range(1,j):
    
    print('------------------------')
    print('petal-GT_number:{0:d}'.format(result_num))
    print('color code    :({0:d},{1:d},{2:d})'.format(int(color_code_GT[result_num,2]),int(color_code_GT[result_num,1]),int(color_code_GT[result_num,0])))
    print('***')
    
    temp = np.zeros(70)
    cnt = 0
    
    for y in range(height):
        for x in range(width):
            if(code_temp_GT[y,x] == result_num):
                if(code_temp_re[y,x]>0):
                    temp[int(code_temp_re[y,x])]+=1
                cnt += 1
                
    petal_max = np.argmax(temp)
    
    if(petal_max>0):
        for y in range(height):
            for x in range(width):
                if(code_temp_re[y,x] == petal_max):
                    img_re[y,x] = color_code_GT[int(result_num)]
    
    flag = 0
    
    for num in range(70):
        if(temp[num]>0):
            rate = (temp[num]/cnt)*100
            print('petal-re_number:{0:d} -> count:{1:d}, {2:.1f}%'.format(num,int(temp[num]),rate))
            #print('petal_max:{0:d}'.format(petal_max))
            flag += 1
        
    if(flag==0):
        print('No matching petals.')
        
cv2.imwrite("dataset/test/ORA-{0:03d}_test.png".format(img_num),img_re)

------------------------
petal-GT_number:1
color code    :(147,128,251)
***
petal-re_number:1 -> count:3362, 99.1%
petal_max:1
------------------------
petal-GT_number:2
color code    :(169,42,57)
***
petal-re_number:2 -> count:189, 9.9%
petal_max:7
petal-re_number:3 -> count:250, 13.1%
petal_max:7
petal-re_number:7 -> count:431, 22.6%
petal_max:7
------------------------
petal-GT_number:3
color code    :(140,148,43)
***
petal-re_number:4 -> count:1157, 98.2%
petal_max:4
------------------------
petal-GT_number:4
color code    :(89,224,216)
***
petal-re_number:5 -> count:2584, 64.9%
petal_max:5
petal-re_number:19 -> count:428, 10.8%
petal_max:5
------------------------
petal-GT_number:5
color code    :(119,65,207)
***
petal-re_number:6 -> count:3937, 90.1%
petal_max:6
petal-re_number:25 -> count:155, 3.5%
petal_max:6
------------------------
petal-GT_number:6
color code    :(72,249,4)
***
petal-re_number:10 -> count:1277, 75.7%
petal_max:10
------------------------
petal-GT_number:7
co

True

In [6]:
code_temp_re[507,373]

21.0

In [7]:
color_code_GT[17]

array([ 50., 132., 232.])

### 参考

In [ ]:
img_num = 700

#input_dataset  = "dataset/annotation_part/ORA-{0:03d}_mask_part.png".format(img_num)
input_dataset  = "dataset/annotation_NLMD/ORA-{0:03d}_mask_NLMD.png".format(img_num)
input_result   = "Output/3D_inte/3D-inte_graph_Trans_reverse/ORA-{0:03d}_3D-inte_graph_Trans_reverse.png".format(img_num)

input_original = "../../flower_CT_photo/ORA/[vg-data] ORA/volume_1/ORA-{0:03d}.tif".format(img_num)

img    = cv2.imread(input_original,0)
img_re = cv2.imread(input_result)
img_GT = cv2.imread(input_dataset)

img_NLMD = cv2.fastNlMeansDenoising(img,h=6)
th_img = np.where(img_NLMD>50 ,255,0)

height,width = img.shape

#花弁の番号の割り当て
color_code_GT = np.zeros((200,3))
color_code    = np.zeros((200,3))

code_temp_GT = np.zeros((height,width))
code_temp_re = np.zeros((height,width))

i = 1
for y in range(height):
    for x in range(width):
        if((img_re[y,x,0]== img_re[y,x,1]) and (img_re[y,x,1] == img_re[y,x,2])):
            img_re[y,x] = (0,0,0)
        else:
            if(i==1):
                color_code[i] = img_re[y,x]
                code_temp_re[y,x] = i
                i+=1

            else:
                flag = 0
                i_cnt = 1

                while(flag != 1 and i_cnt <= i):
                    if((color_code[i_cnt]==img_re[y,x]).all()):
                        flag = 1
                        code_temp_re[y,x] = i_cnt
                    else:
                        i_cnt += 1

                if(flag==0):
                    color_code[i] = img_re[y,x]
                    code_temp_re[y,x] = i
                    i+=1
                    
j = 1
for y in range(height):
    for x in range(width):
        if((img_GT[y,x,0]== img_GT[y,x,1]) and (img_GT[y,x,1] == img_GT[y,x,2])):
            img_GT[y,x] = (0,0,0)
        else:
            if(j==1):
                color_code_GT[j] = img_GT[y,x]
                code_temp_GT[y,x] = j
                j+=1

            else:
                flag = 0
                j_cnt = 1

                while(flag != 1 and j_cnt <= j):
                    if((color_code_GT[j_cnt]==img_GT[y,x]).all()):
                        flag = 1
                        code_temp_GT[y,x] = j_cnt
                    else:
                        j_cnt += 1

                if(flag==0):
                    color_code_GT[j] = img_GT[y,x]
                    code_temp_GT[y,x] = j
                    j+=1

#TP,FP,FNの割り当て
for result_num in range(1,i):
    
    print('------------------------')
    print('petal-re_number:{0:d}'.format(result_num))
    print('color code    :({0:d},{1:d},{2:d})'.format(int(color_code[result_num,2]),int(color_code[result_num,1]),int(color_code[result_num,0])))
    print('***')
    
    temp = np.zeros(70)
    cnt = 0
    
    for y in range(height):
        for x in range(width):
            if(code_temp_re[y,x] == result_num):
                temp[int(code_temp_GT[y,x])]+=1
                cnt += 1
    
    for num in range(70):
        if(temp[num]>0):
            rate = (temp[num]/cnt)*100
            print('petal-GT_number:{0:d} -> count:{1:d}, {2:.1f}'.format(num,int(temp[num]),rate))